<a href="https://colab.research.google.com/github/andrebododea/TinyML/blob/master/tinyml_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Colab with GitHub
## Doing it
